In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

import sys
sys.path.append('/Users/mads/Documents/Python/kermut/')
from src.experiments.investigate_correlations import load_protein_mpnn_outputs


In [2]:
import torch
import torch.nn as nn
from gpytorch.kernels import Kernel

from src.experiments.investigate_correlations import load_protein_mpnn_outputs
from src.model.utils import Tokenizer
from src import GFP_WT, BLAT_ECOLX_WT

In [3]:
from src.model.kernel import KermutJSKernel, KermutJSD_RBFKernel, KermutHellingerKernel, KermutHellingerKernelMulti

In [7]:
dataset = "GFP"
conditional_probs_path = Path(
    '../',
    "data",
    "interim",
    dataset,
    "proteinmpnn",
    "conditional_probs_only",
    f"{dataset}.npz",
)
assay_path = Path('../', "data", "processed", f"{dataset}.tsv")

# Load data
conditional_probs = load_protein_mpnn_outputs(
    conditional_probs_path, as_tensor=True
)
df = pd.read_csv(assay_path, sep="\t")
df = df[df["n_muts"] <= 2]
df = df.iloc[:1000]
y = df["delta_fitness"].values
y = torch.tensor(y)

tokenizer = Tokenizer()
sequences = df["seq"]
tokens = tokenizer(sequences)
wt_sequence = tokenizer([GFP_WT])[0]

model_kwargs = {
    "conditional_probs": conditional_probs,
    "wt_sequence": wt_sequence,
    "p_B": 15.0,
    "p_Q": 5.0,
    "theta": 1.0,
    "gamma": 1.0,
    "learnable_transform": False,
    "learnable_hellinger": False,
}

kernel = KermutHellingerKernelMulti(**model_kwargs)

# FOR PROFILING
# from torch.profiler import profile, record_function, ProfilerActivity

# with profile(
#     activities=[ProfilerActivity.CPU], profile_memory=True, record_shapes=True
# ) as prof:
#     with record_function("model_inference"):
out = kernel(tokens)  # IF PROFILING, INDENT

# print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))
# print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))

FileNotFoundError: [Errno 2] No such file or directory: '../src/data/interim/GFP/proteinmpnn/conditional_probs_only/GFP.npz'